<a href="https://colab.research.google.com/github/olonok69/LLM_Notebooks/blob/main/mlflow/evaluation/LLM_Evaluation_with_MLflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM Evaluation Metrics
1. There are two types of LLM evaluation metrics in MLflow:

Metrics relying on SaaS model (e.g., OpenAI) for scoring, e.g., mlflow.metrics.genai.answer_relevance(). These metrics are created via mlflow.metrics.genai.make_genai_metric() method. For each data record, these metrics under the hood sends one prompt consisting of the following information to the SaaS model, and extract the score from model response:

- Metrics definition.

- Metrics grading criteria.

- Reference examples.

- Input data/context.

- Model output.

- [optional] Ground truth.

More details of how these fields are set can be found in the section “Create your Custom LLM-evaluation Metrics”.

2. Function-based per-row metrics. These metrics calculate a score for each data record (row in terms of Pandas/Spark dataframe), based on certain functions, like Rouge (mlflow.metrics.rougeL()) or Flesch Kincaid (mlflow.metrics.flesch_kincaid_grade_level()). These metrics are similar to traditional metrics.



In [1]:
%pip install "mlflow==2.12.1" "llama-index>=0.10.44"  pyngrok openai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.8/176.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/

In [2]:
! pip install  evaluate torch textstat  -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from pyngrok import ngrok

get_ipython().system_raw("mlflow ui --port 5000 &")


# Terminate open tunnels if exist
ngrok.kill()

In [4]:

from google.colab import userdata
NGROK_AUTH_TOKEN  = userdata.get('NGROK')

ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

MLflow Tracking UI: https://b2c7-34-85-252-58.ngrok-free.app


In [5]:
import openai
import pandas as pd

import mlflow
import os

from google.colab import userdata
import pprint

os.environ["OPENAI_API_KEY"] = userdata.get('KEY_OPENAI')

In [6]:
assert "OPENAI_API_KEY" in os.environ, "Please set the OPENAI_API_KEY environment variable."

# Basic Question-Answering Evaluation

Create a test case of inputs that will be passed into the model and ground_truth which will be used to compare against the generated output from the model.

- https://huggingface.co/spaces/evaluate-measurement/toxicity
- https://en.wikipedia.org/wiki/Automated_readability_index
- https://en.wikipedia.org/wiki/Flesch%E2%80%93Kincaid_readability_tests#Flesch%E2%80%93Kincaid_grade_level


# Toxicity
https://huggingface.co/facebook/roberta-hate-speech-dynabench-r4-target

# Textstat
Textstat is an easy to use library to calculate statistics from text. It helps determine readability, complexity, and grade level.

https://pypi.org/project/textstat/

## Function-based per-row metrics.
### question-answering: model_type="question-answering":

- exact-match

- toxicity

- ari_grade_level

- flesch_kincaid_grade_level

### text-summarization: model_type="text-summarization":

- ROUGE

- toxicity

- ari_grade_level

- flesch_kincaid_grade_level

### text models: model_type="text":

- toxicity

- ari_grade_level

- flesch_kincaid_grade_level

### retrievers: model_type="retriever":

- precision_at_k

- recall_at_k

- ndcg_at_k



In [11]:
eval_df = pd.DataFrame(
    {
        "inputs": [
            "How does useEffect() work?",
            "What does the static keyword in a function mean?",
            "What does the 'finally' block in Python do?",
            "What is the difference between multiprocessing and multithreading?",
        ],
        "ground_truth": [
            "The useEffect() hook tells React that your component needs to do something after render. React will remember the function you passed (we’ll refer to it as our “effect”), and call it later after performing the DOM updates.",
            "Static members belongs to the class, rather than a specific instance. This means that only one instance of a static member exists, even if you create multiple objects of the class, or if you don't create any. It will be shared by all objects.",
            "'Finally' defines a block of code to run when the try... except...else block is final. The finally block will be executed no matter if the try block raises an error or not.",
            "Multithreading refers to the ability of a processor to execute multiple threads concurrently, where each thread runs a process. Whereas multiprocessing refers to the ability of a system to run multiple processors in parallel, where each processor can run one or more threads.",
        ],
    }
)

In [12]:
mlflow.set_tracking_uri("http://localhost:5000")
#
mlflow.set_experiment("Evaluate LLM,s")

<Experiment: artifact_location='mlflow-artifacts:/771864919623380882', creation_time=1731358712851, experiment_id='771864919623380882', last_update_time=1731358712851, lifecycle_stage='active', name='Evaluate LLM,s', tags={}>

In [37]:
# help(mlflow.metrics)

In [13]:
with mlflow.start_run() as run:
    system_prompt = "Answer the following question in two sentences"
    basic_qa_model = mlflow.openai.log_model(
        model="gpt-4o-mini",
        task=openai.chat.completions,
        artifact_path="model",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "{question}"},
        ],
    )
    results = mlflow.evaluate(
        basic_qa_model.model_uri,
        eval_df,
        targets="ground_truth",  # specify which column corresponds to the expected output
        model_type="question-answering",  # model type indicates which metrics are relevant for this task
        evaluators="default",
    )
results.metrics

/usr/local/lib/python3.10/dist-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  string_columns = trimmed_df.columns[(df.applymap(type) == str).all(0)]
/usr/local/lib/python3.10/dist-packages/mlflow/models/evaluation/base.py:521: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(_hash_array_like_element_as_bytes)
2024/11/11 21:43:53 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2024/11/11 21:43:53 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/11/11 21:43:55 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...


{'toxicity/v1/mean': 0.00021652834402630106,
 'toxicity/v1/variance': 2.0613665907112866e-09,
 'toxicity/v1/p90': 0.00026491407916182655,
 'toxicity/v1/ratio': 0.0,
 'flesch_kincaid_grade_level/v1/mean': 14.075,
 'flesch_kincaid_grade_level/v1/variance': 6.536874999999999,
 'flesch_kincaid_grade_level/v1/p90': 16.560000000000002,
 'ari_grade_level/v1/mean': 17.875,
 'ari_grade_level/v1/variance': 8.646875000000003,
 'ari_grade_level/v1/p90': 21.14,
 'exact_match/v1': 0.0}

In [14]:
results.tables["eval_results_table"]

,inputs,ground_truth,outputs,token_count,toxicity/v1/score,flesch_kincaid_grade_level/v1/score,ari_grade_level/v1/score
0,How does useEffect() work?,The useEffect() hook tells React that your com...,The `useEffect()` hook in React allows you to ...,78,0.000287,16.0,19.6
1,What does the static keyword in a function mean?,"Static members belongs to the class, rather th...",The `static` keyword in a function means that ...,49,0.000205,10.3,14.7
2,What does the 'finally' block in Python do?,'Finally' defines a block of code to run when ...,The 'finally' block in Python is used to defin...,64,0.000214,13.2,15.4
3,What is the difference between multiprocessing...,Multithreading refers to the ability of a proc...,Multiprocessing involves running multiple proc...,62,0.000160,16.8,21.8


# LLM-judged correctness with OpenAI GPT-4

### Metrics with LLM as the Judge
MLflow offers a few pre-canned metrics which uses LLM as the judge. Despite the difference under the hood, the usage is the same - put these metrics in the extra_metrics argument in mlflow.evaluate(). Here is the list of pre-canned metrics:

- mlflow.metrics.genai.answer_similarity(): Use this metric when you want to evaluate how similar the model generated output is compared to the information in the ground_truth. High scores mean that your model outputs contain similar information as the ground_truth, while low scores mean that outputs may disagree with the ground_truth.

- mlflow.metrics.genai.answer_correctness(): Use this metric when you want to evaluate how factually correct the model generated output is based on the information in the ground_truth. High scores mean that your model outputs contain similar information as the ground_truth and that this information is correct, while low scores mean that outputs may disagree with the ground_truth or that the information in the output is incorrect. Note that this builds onto answer_similarity.

- mlflow.metrics.genai.answer_relevance(): Use this metric when you want to evaluate how relevant the model generated output is to the input (context is ignored). High scores mean that your model outputs are about the same subject as the input, while low scores mean that outputs may be non-topical.

- mlflow.metrics.genai.relevance(): Use this metric when you want to evaluate how relevant the model generated output is with respect to both the input and the context. High scores mean that the model has understood the context and correct extracted relevant information from the context, while low score mean that output has completely ignored the question and the context and could be hallucinating.

- mlflow.metrics.genai.faithfulness(): Use this metric when you want to evaluate how faithful the model generated output is based on the context provided. High scores mean that the outputs contain information that is in line with the context, while low scores mean that outputs may disagree with the context (input is ignored).


- https://mlflow.org/docs/latest/llms/llm-evaluate/index.html
- https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.evaluate

```
mlflow.metrics.genai.__all__

['EvaluationExample',
 'make_genai_metric',
 'answer_similarity',
 'answer_correctness',
 'faithfulness',
 'answer_relevance',
 'relevance']
 ```

In [15]:
from mlflow.metrics.genai import EvaluationExample, answer_similarity, answer_correctness, faithfulness, answer_relevance, relevance

In [21]:
#help(mlflow.metrics.genai)

In [16]:
help(answer_similarity)

Help on function answer_similarity in module mlflow.metrics.genai.metric_definitions:

answer_similarity(model: Optional[str] = None, metric_version: Optional[str] = None, examples: Optional[List[mlflow.metrics.genai.base.EvaluationExample]] = None) -> mlflow.models.evaluation.base.EvaluationMetric
    .. Note:: Experimental: This function may change or be removed in a future release without warning.
    
    
    This function will create a genai metric used to evaluate the answer similarity of an LLM
    using the model provided. Answer similarity will be assessed by the semantic similarity of the
    output to the ``ground_truth``, which should be specified in the ``targets`` column.
    
    The ``targets`` eval_arg must be provided as part of the input dataset or output
    predictions. This can be mapped to a column of a different name using ``col_mapping``
    in the ``evaluator_config`` parameter, or using the ``targets`` parameter in mlflow.evaluate().
    
    An MlflowExcept

In [19]:
from mlflow.metrics.genai import EvaluationExample, answer_similarity

# Create an example to describe what answer_similarity means like for this problem.
example = EvaluationExample(
    input="What is MLflow?",
    output="MLflow is an open-source platform for managing machine "
    "learning workflows, including experiment tracking, model packaging, "
    "versioning, and deployment, simplifying the ML lifecycle.",
    score=4,
    justification="The definition effectively explains what MLflow is "
    "its purpose, and its developer. It could be more concise for a 5-score.",
    grading_context={
        "targets": "MLflow is an open-source platform for managing "
        "the end-to-end machine learning (ML) lifecycle. It was developed by Databricks, "
        "a company that specializes in big data and machine learning solutions. MLflow is "
        "designed to address the challenges that data scientists and machine learning "
        "engineers face when developing, training, and deploying machine learning models."
    },
)

# Construct the metric using OpenAI GPT-4 as the judge
answer_similarity_metric = answer_similarity(model="openai:/gpt-4", examples=[example])

print(answer_similarity_metric)

EvaluationMetric(name=answer_similarity, greater_is_better=True, long_name=answer_similarity, version=v1, metric_details=
Task:
You must return the following fields in your response in two lines, one below the other:
score: Your numerical score for the model's answer_similarity based on the rubric
justification: Your reasoning about the model's answer_similarity score

You are an impartial judge. You will be given an input that was sent to a machine
learning model, and you will be given an output that the model produced. You
may also be given additional information that was used by the model to generate the output.

Your task is to determine a numerical score called answer_similarity based on the input and output.
A definition of answer_similarity and a grading rubric are provided below.
You must use the grading rubric to determine your score. You must also justify your score.

Examples could be included below for reference. Make sure to use them as references and to
understand them be

In [20]:
from mlflow.metrics.genai import EvaluationExample, make_genai_metric
# The definition explains what "answer quality" entails
answer_quality_definition = """Please evaluate answer quality for the provided output on the following criteria:
fluency, clarity, and conciseness. Each of the criteria is defined as follows:
  - Fluency measures how naturally and smooth the output reads.
  - Clarity measures how understandable the output is.
  - Conciseness measures the brevity and efficiency of the output without compromising meaning.
The more fluent, clear, and concise a text, the higher the score it deserves.
"""

# The grading prompt explains what each possible score means
answer_quality_grading_prompt = """Answer quality: Below are the details for different scores:
  - Score 1: The output is entirely incomprehensible and cannot be read.
  - Score 2: The output conveys some meaning, but needs lots of improvement in to improve fluency, clarity, and conciseness.
  - Score 3: The output is understandable but still needs improvement.
  - Score 4: The output performs well on two of fluency, clarity, and conciseness, but could be improved on one of these criteria.
  - Score 5: The output reads smoothly, is easy to understand, and clear. There is no clear way to improve the output on these criteria.
"""

# We provide an example of a "bad" output
example1 = EvaluationExample(
    input="What is MLflow?",
    output="MLflow is an open-source platform. For managing machine learning workflows, it "
    "including experiment tracking model packaging versioning and deployment as well as a platform "
    "simplifying for on the ML lifecycle.",
    score=2,
    justification="The output is difficult to understand and demonstrates extremely low clarity. "
    "However, it still conveys some meaning so this output deserves a score of 2.",
)

# We also provide an example of a "good" output
example2 = EvaluationExample(
    input="What is MLflow?",
    output="MLflow is an open-source platform for managing machine learning workflows, including "
    "experiment tracking, model packaging, versioning, and deployment.",
    score=5,
    justification="The output is easily understandable, clear, and concise. It deserves a score of 5.",
)

answer_quality_metric = make_genai_metric(
    name="answer_quality",
    definition=answer_quality_definition,
    grading_prompt=answer_quality_grading_prompt,
    version="v1",
    examples=[example1, example2],
    model="openai:/gpt-4",
    greater_is_better=True,
)
print(answer_quality_metric)

EvaluationMetric(name=answer_quality, greater_is_better=True, long_name=answer_quality, version=v1, metric_details=
Task:
You must return the following fields in your response in two lines, one below the other:
score: Your numerical score for the model's answer_quality based on the rubric
justification: Your reasoning about the model's answer_quality score

You are an impartial judge. You will be given an input that was sent to a machine
learning model, and you will be given an output that the model produced. You
may also be given additional information that was used by the model to generate the output.

Your task is to determine a numerical score called answer_quality based on the input and output.
A definition of answer_quality and a grading rubric are provided below.
You must use the grading rubric to determine your score. You must also justify your score.

Examples could be included below for reference. Make sure to use them as references and to
understand them before completing th

In [21]:
with mlflow.start_run() as run:
    results = mlflow.evaluate(
        basic_qa_model.model_uri,
        eval_df,
        targets="ground_truth",
        model_type="question-answering",
        evaluators="default",
        extra_metrics=[answer_similarity_metric, answer_quality_metric],  # use the answer similarity metric created above
    )
results.metrics



/usr/local/lib/python3.10/dist-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  string_columns = trimmed_df.columns[(df.applymap(type) == str).all(0)]
/usr/local/lib/python3.10/dist-packages/mlflow/models/evaluation/base.py:521: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(_hash_array_like_element_as_bytes)
2024/11/11 21:55:12 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2024/11/11 21:55:12 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/11/11 21:55:14 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

{'toxicity/v1/mean': 0.00022456275473814458,
 'toxicity/v1/variance': 2.13792246050415e-09,
 'toxicity/v1/p90': 0.0002727492479607463,
 'toxicity/v1/ratio': 0.0,
 'flesch_kincaid_grade_level/v1/mean': 14.399999999999999,
 'flesch_kincaid_grade_level/v1/variance': 0.34000000000000025,
 'flesch_kincaid_grade_level/v1/p90': 15.040000000000001,
 'ari_grade_level/v1/mean': 17.425,
 'ari_grade_level/v1/variance': 0.761875000000001,
 'ari_grade_level/v1/p90': 18.38,
 'exact_match/v1': 0.0,
 'answer_similarity/v1/mean': 3.75,
 'answer_similarity/v1/variance': 1.1875,
 'answer_similarity/v1/p90': 4.7,
 'answer_quality/v1/mean': 5.0,
 'answer_quality/v1/variance': 0.0,
 'answer_quality/v1/p90': 5.0}

In [23]:
results.tables["eval_results_table"]



,inputs,ground_truth,outputs,token_count,toxicity/v1/score,flesch_kincaid_grade_level/v1/score,ari_grade_level/v1/score,answer_similarity/v1/score,answer_similarity/v1/justification,answer_quality/v1/score,answer_quality/v1/justification
0,How does useEffect() work?,The useEffect() hook tells React that your com...,The `useEffect()` hook in React allows you to ...,66,0.000277,15.4,17.1,4,The output accurately describes the `useEffect...,5,"The output is fluent, clear, and concise. It p..."
1,What does the static keyword in a function mean?,"Static members belongs to the class, rather th...",The `static` keyword in a function means that ...,57,0.000187,14.0,17.4,2,The output provides a correct explanation of t...,5,"The output is fluent, clear, and concise. It a..."
2,What does the 'finally' block in Python do?,'Finally' defines a block of code to run when ...,The 'finally' block in Python is used to defin...,67,0.000264,14.0,16.4,5,The output closely aligns with the provided ta...,5,"The model's response is fluent, clear, and con..."
3,What is the difference between multiprocessing...,Multithreading refers to the ability of a proc...,Multiprocessing involves using multiple proces...,67,0.000171,14.2,18.8,4,The output accurately describes the concepts o...,5,"The model's output is fluent, clear, and conci..."


In [24]:
from mlflow.metrics.genai import EvaluationExample, make_genai_metric

professionalism_metric = make_genai_metric(
  name="professionalism",
    definition=(
        "Professionalism refers to the use of a formal, respectful, and appropriate style of communication that is tailored to the context and audience. It often involves avoiding overly casual language, slang, or colloquialisms, and instead using clear, concise, and respectful language"
    ),
    grading_prompt=(
        "Professionalism: If the answer is written using a professional tone, below "
        "are the details for different scores: "
        "- Score 1: Language is extremely casual, informal, and may include slang or colloquialisms. Not suitable for professional contexts."
        "- Score 2: Language is casual but generally respectful and avoids strong informality or slang. Acceptable in some informal professional settings."
        "- Score 3: Language is balanced and avoids extreme informality or formality. Suitable for most professional contexts. "
        "- Score 4: Language is noticeably formal, respectful, and avoids casual elements. Appropriate for business or academic settings. "
        "- Score 5: Language is excessively formal, respectful, and avoids casual elements. Appropriate for the most formal settings such as textbooks. "
    ),
    examples=[
        EvaluationExample(
            input="What is MLflow?",
            output=(
                "MLflow is like your friendly neighborhood toolkit for managing your machine learning projects. It helps you track experiments, package your code and models, and collaborate with your team, making the whole ML workflow smoother. It's like your Swiss Army knife for machine learning!"
            ),
            score=2,
            justification=(
                "The response is written in a casual tone. It uses contractions, filler words such as 'like', and exclamation points, which make it sound less professional. "
            ),
        )
    ],
    version="v1",
    model="openai:/gpt-4o",
    parameters={"temperature": 0.0},
    grading_context_columns=[],
    aggregations=["mean", "variance", "p90"],
    greater_is_better=True,
)

print(professionalism_metric)

EvaluationMetric(name=professionalism, greater_is_better=True, long_name=professionalism, version=v1, metric_details=
Task:
You must return the following fields in your response in two lines, one below the other:
score: Your numerical score for the model's professionalism based on the rubric
justification: Your reasoning about the model's professionalism score

You are an impartial judge. You will be given an input that was sent to a machine
learning model, and you will be given an output that the model produced. You
may also be given additional information that was used by the model to generate the output.

Your task is to determine a numerical score called professionalism based on the input and output.
A definition of professionalism and a grading rubric are provided below.
You must use the grading rubric to determine your score. You must also justify your score.

Examples could be included below for reference. Make sure to use them as references and to
understand them before complet

In [25]:
with mlflow.start_run() as run:
    results = mlflow.evaluate(
        basic_qa_model.model_uri,
        eval_df,
        model_type="question-answering",
        evaluators="default",
        extra_metrics=[professionalism_metric],  # use the professionalism metric we created above
    )
print(results.metrics)

/usr/local/lib/python3.10/dist-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  string_columns = trimmed_df.columns[(df.applymap(type) == str).all(0)]
/usr/local/lib/python3.10/dist-packages/mlflow/models/evaluation/base.py:521: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(_hash_array_like_element_as_bytes)
2024/11/11 22:05:17 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2024/11/11 22:05:17 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/11/11 22:05:20 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/11/11 22:05:20 WARNING mlflow.models.evaluation.default_evaluator: Did not log builtin metric 'exact_match' because it returned None.


  0%|          | 0/1 [00:00<?, ?it/s]

2024/11/11 22:05:22 WARNING mlflow.models.evaluation.default_evaluator: Did not log builtin metric 'exact_match' because it returned None.


  0%|          | 0/4 [00:00<?, ?it/s]

{'toxicity/v1/mean': 0.00019768811034737155, 'toxicity/v1/variance': 2.1561239419315333e-09, 'toxicity/v1/p90': 0.00024717401975067333, 'toxicity/v1/ratio': 0.0, 'flesch_kincaid_grade_level/v1/mean': 13.625, 'flesch_kincaid_grade_level/v1/variance': 8.226874999999996, 'flesch_kincaid_grade_level/v1/p90': 16.85, 'ari_grade_level/v1/mean': 16.3, 'ari_grade_level/v1/variance': 12.255000000000003, 'ari_grade_level/v1/p90': 20.150000000000002, 'professionalism/v1/mean': 3.75, 'professionalism/v1/variance': 0.1875, 'professionalism/v1/p90': 4.0}


In [26]:
results.tables["eval_results_table"]

,inputs,ground_truth,outputs,token_count,toxicity/v1/score,flesch_kincaid_grade_level/v1/score,ari_grade_level/v1/score,professionalism/v1/score,professionalism/v1/justification
0,How does useEffect() work?,The useEffect() hook tells React that your com...,The `useEffect()` hook in React allows you to ...,65,0.000254,14.4,16.3,3,The response uses a balanced tone that is clea...
1,What does the static keyword in a function mean?,"Static members belongs to the class, rather th...",The static keyword in a function indicates tha...,55,0.000140,11.9,14.9,4,The response is written in a formal and respec...
2,What does the 'finally' block in Python do?,'Finally' defines a block of code to run when ...,The 'finally' block in Python is used to defin...,54,0.000231,10.3,12.2,4,The response is written in a formal and respec...
3,What is the difference between multiprocessing...,Multithreading refers to the ability of a proc...,Multiprocessing involves running multiple proc...,66,0.000166,17.9,21.8,4,The response is written in a formal and respec...


In [27]:
with mlflow.start_run() as run:
    system_prompt = "Answer the following question using extreme formality."
    professional_qa_model = mlflow.openai.log_model(
        model="gpt-4o-mini",
        task=openai.chat.completions,
        artifact_path="model",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "{question}"},
        ],
    )
    results = mlflow.evaluate(
        professional_qa_model.model_uri,
        eval_df,
        model_type="question-answering",
        evaluators="default",
        extra_metrics=[professionalism_metric],
    )
print(results.metrics)

/usr/local/lib/python3.10/dist-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  string_columns = trimmed_df.columns[(df.applymap(type) == str).all(0)]
/usr/local/lib/python3.10/dist-packages/mlflow/models/evaluation/base.py:521: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(_hash_array_like_element_as_bytes)
2024/11/11 22:08:27 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2024/11/11 22:08:27 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/11/11 22:08:34 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/11/11 22:08:35 WARNING mlflow.models.evaluation.default_evaluator: Did not log builtin metric 'exact_match' because it returned None.


  0%|          | 0/1 [00:00<?, ?it/s]

2024/11/11 22:08:40 WARNING mlflow.models.evaluation.default_evaluator: Did not log builtin metric 'exact_match' because it returned None.


  0%|          | 0/4 [00:00<?, ?it/s]

{'toxicity/v1/mean': 0.0005983808150631376, 'toxicity/v1/variance': 2.757988855937279e-07, 'toxicity/v1/p90': 0.0011565125547349454, 'toxicity/v1/ratio': 0.0, 'flesch_kincaid_grade_level/v1/mean': 15.9, 'flesch_kincaid_grade_level/v1/variance': 4.075000000000003, 'flesch_kincaid_grade_level/v1/p90': 18.17, 'ari_grade_level/v1/mean': 18.525, 'ari_grade_level/v1/variance': 3.6368750000000007, 'ari_grade_level/v1/p90': 20.38, 'professionalism/v1/mean': 4.75, 'professionalism/v1/variance': 0.1875, 'professionalism/v1/p90': 5.0}


In [28]:
results.tables["eval_results_table"]

,inputs,ground_truth,outputs,token_count,toxicity/v1/score,flesch_kincaid_grade_level/v1/score,ari_grade_level/v1/score,professionalism/v1/score,professionalism/v1/justification
0,How does useEffect() work?,The useEffect() hook tells React that your com...,The `useEffect` hook is a fundamental feature ...,396,0.001507,13.7,15.6,4,The response is written in a noticeably formal...
1,What does the static keyword in a function mean?,"Static members belongs to the class, rather th...","The designation of the keyword ""static"" within...",185,0.000339,14.8,18.3,5,The response is written in an excessively form...
2,What does the 'finally' block in Python do?,'Finally' defines a block of code to run when ...,"In the esteemed realm of Python programming, t...",227,0.000268,16.0,19.4,5,The response is written in an excessively form...
3,What is the difference between multiprocessing...,Multithreading refers to the ability of a proc...,The distinction between multiprocessing and mu...,274,0.000280,19.1,20.8,5,The response is written in an excessively form...
